#universidad nacional de san antonio abad del cusco


*   Nombre: JHON EDWIN TACUSI LAROTA
*   Codigo: 150408
*   Segundo examen parcial de mineria de datos



In [2]:
!pip install pyspark
!pip install pyspark==3.0.1 py4j==0.10.9

     |████████████████████████████████| 281.3 MB 37 kB/s 
     |████████████████████████████████| 198 kB 57.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=084c7bc4ea27712f76ad5bacc385f6ec5abe45c8d041fb46e10830f9fd84be66
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark
     |████████████████████████████████| 204.2 MB 33 kB/s 
     |████████████████████████████████| 198 kB 57.5 MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612246 sha256=81acafe8e2b380fee048cda1196af7873f6551cdad0638a50ea83c186f9ccaa7
  Stored in directory: /root/.cache/pip/wheels/5e/34/fa/b37b5cef503fc5148b478b2495043ba61b079120b7ff379f9b
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.2
    Uninstalling py4j-0.10.9.2:
      Successfully uninstalled py4j-0.10.9.2
  Attempting uninstall: pys



#EJERCICIO Implementar en un notebook BoW pyspark usando mapreduce el algoritmo TF-IDF

In [3]:
import math
from pyspark.sql.functions import *
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
sc =SparkContext.getOrCreate()
import os
import re
import nltk.corpus
from nltk import corpus


#Frecuencia de término de cómputo (TF)
La frecuencia del término es el número de veces que aparece una palabra en particular en un documento. Primero, creemos un conjunto de datos de juguetes de la siguiente manera:



*   primeramente usaremos un ejemplo para la implementacion de tf y idf
*   realizamos la tokenizacion



In [4]:

# Como prueba de ejemplo utilizamos un data con 3 documentos
data=[(1,'la escuela de informatica'),(2,"yo amo a mi facultad"),(3,"la mejor facultad de la universidad")]
lines=sc.parallelize(data)
lines.collect()
map1=lines.flatMap(lambda x: [((x[0],i),1) for i in x[1].split()])
map1.collect()
reduce=map1.reduceByKey(lambda x,y:x+y)
reduce.collect()

[((1, 'la'), 1),
 ((1, 'escuela'), 1),
 ((1, 'de'), 1),
 ((1, 'informatica'), 1),
 ((2, 'yo'), 1),
 ((2, 'mi'), 1),
 ((3, 'la'), 2),
 ((3, 'facultad'), 1),
 ((3, 'de'), 1),
 ((2, 'amo'), 1),
 ((2, 'a'), 1),
 ((2, 'facultad'), 1),
 ((3, 'mejor'), 1),
 ((3, 'universidad'), 1)]



*   implementamos el codigo para obtener el TF de nuestro ejemplo
*   mostramos el resultado obtenido



In [5]:

#=================================================================
# Algoritmo termino de frecuencia (TF)
tf=reduce.map(lambda x: (x[0][1],(x[0][0],x[1])))
tf.collect()

map3=reduce.map(lambda x: (x[0][1],(x[0][0],x[1],1)))
map3.collect()

map4=map3.map(lambda x:(x[0],x[1][2]))
map4.collect()

reduce2=map4.reduceByKey(lambda x,y:x+y)
reduce2.collect()


[('yo', 1),
 ('mi', 1),
 ('mejor', 1),
 ('universidad', 1),
 ('la', 2),
 ('escuela', 1),
 ('de', 2),
 ('informatica', 1),
 ('facultad', 2),
 ('amo', 1),
 ('a', 1)]



*   implementamos el codigo oara obtener el IDF de nuestro ejemplo
*   mostramos el resultados obtenido



In [11]:


#=================================================================
# Algoritmo termino de frecuencia (IDF)
idf=reduce2.map(lambda x: (x[0],math.log10(len(data)/x[1])))
idf.collect()

rdd=tf.join(idf)
rdd.collect()


[('yo', ((2, 1), 0.47712125471966244)),
 ('mejor', ((3, 1), 0.47712125471966244)),
 ('universidad', ((3, 1), 0.47712125471966244)),
 ('la', ((1, 1), 0.17609125905568124)),
 ('la', ((3, 2), 0.17609125905568124)),
 ('escuela', ((1, 1), 0.47712125471966244)),
 ('informatica', ((1, 1), 0.47712125471966244)),
 ('a', ((2, 1), 0.47712125471966244)),
 ('mi', ((2, 1), 0.47712125471966244)),
 ('de', ((1, 1), 0.17609125905568124)),
 ('de', ((3, 1), 0.17609125905568124)),
 ('facultad', ((3, 1), 0.17609125905568124)),
 ('facultad', ((2, 1), 0.17609125905568124)),
 ('amo', ((2, 1), 0.47712125471966244))]



*   implementamo el codigo para TF-IDF
*   Mostramos en una tabla los resultados obtenidos



In [12]:

#=================================================================
# Algoritmo termino de frecuencia (TF-IDF)
rdd=rdd.map(lambda x: (x[1][0][0],(x[0],x[1][0][1],x[1][1],x[1][0][1]*x[1][1]))).sortByKey()
rdd.collect()
# se muestra el ID del documento, token , TF, IDF y TF-IDF


[(1, ('la', 1, 0.17609125905568124, 0.17609125905568124)),
 (1, ('escuela', 1, 0.47712125471966244, 0.47712125471966244)),
 (1, ('informatica', 1, 0.47712125471966244, 0.47712125471966244)),
 (1, ('de', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('yo', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('a', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('mi', 1, 0.47712125471966244, 0.47712125471966244)),
 (2, ('facultad', 1, 0.17609125905568124, 0.17609125905568124)),
 (2, ('amo', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('mejor', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('universidad', 1, 0.47712125471966244, 0.47712125471966244)),
 (3, ('la', 2, 0.17609125905568124, 0.3521825181113625)),
 (3, ('de', 1, 0.17609125905568124, 0.17609125905568124)),
 (3, ('facultad', 1, 0.17609125905568124, 0.17609125905568124))]

In [13]:
spark = SparkSession(sc)
rdd=rdd.map(lambda x: (x[0],x[1][0],x[1][1],x[1][2],x[1][3]))
hasattr(rdd, "toDF")
rdd.toDF(["ID del Documento","Token","TF","IDF","TF-IDF"]).show()

+----------------+-----------+---+-------------------+-------------------+
|ID del Documento|      Token| TF|                IDF|             TF-IDF|
+----------------+-----------+---+-------------------+-------------------+
|               1|         la|  1|0.17609125905568124|0.17609125905568124|
|               1|    escuela|  1|0.47712125471966244|0.47712125471966244|
|               1|informatica|  1|0.47712125471966244|0.47712125471966244|
|               1|         de|  1|0.17609125905568124|0.17609125905568124|
|               2|         yo|  1|0.47712125471966244|0.47712125471966244|
|               2|          a|  1|0.47712125471966244|0.47712125471966244|
|               2|         mi|  1|0.47712125471966244|0.47712125471966244|
|               2|   facultad|  1|0.17609125905568124|0.17609125905568124|
|               2|        amo|  1|0.47712125471966244|0.47712125471966244|
|               3|      mejor|  1|0.47712125471966244|0.47712125471966244|
|               3|univers